<center>
![alt text](http://i64.tinypic.com/as8k4.jpg “Title”)  
## [mlcourse.ai](https://mlcourse.ai/)   Open Machine learning course
</center>
<center>Author:Natalia Domozhirova, slack: @ndomozhirova.

# <center>Tutorial</center>

<center>
![alt text](http://i63.tinypic.com/35mpimt.jpg)

# <center>KERAS : easy way to construct the Neural Networks</center>

Keras is a high-level neural networks API, written in Python.

Major Keras features:
- its  capable of running on top of TensorFlow, CNTK, or Theano.
- Keras allows for easy and fast prototyping and supports both Perceptrons, Convolutional networks and Recurrent networks (including LSTM), as well as their combinations. 
- Keras is compatible with: Python 2.7-3.6.

To make the process more interesting let's consider the classification example  from the real life.

## <center>Example description. 

Let's take the task from one Hakaton, organized by some polypropylene producer this year. So, let’s consider the production of the polypropylene granules by extruder. Extruder is a kind of “meat grinder” which has the knives at the end of the process which are cutting the output product onto granules of the whole process.  
The problem is that sometimes the production mass has an irregular consistency and sticks to the knives. When there is a lot of stuck mass - knives can no longer function. So,  it is necessary to stop production process, which is very expensive. If we catch the very beginning of such sticking process -  there is a  method to very quickly and painless  clean the knives and continue production without stopping.
So, the task is to send the stop signal to operator a bit in advance (let’s say not later then 15 minutes) – so that he could have a time for necessary manipulations.

<center>
<img src= "http://i68.tinypic.com/2rr2glg.jpg
" style="height:250px">      
 

Now we’re considering already preprocessed normalized dataset with the vectors of the system sensor’s values  (5,160 features) and “0” or “1” targets. It is already devided by [train](https://drive.google.com/open?id=1TMlClLguxcXTOAJt8VKe-iLrndJuFShl) and [test](https://drive.google.com/open?id=1JonMu0wmMbUqcbSd17Qr2A3AhVF3nutZ). 
Let's download and prepare to work our datasets. In the datasets there are targets in zero column and the timestamps -in the 1st column.So, let's extract our train and test matrix as well as our targets. Also we'll transform targets to categorical -so to have as a result y 2-dimentional vector, i.e. the vector with probabilities of 0 and 1.

In [1]:
import pandas as pd
import numpy as np
from keras.utils import np_utils

df_train = pd.read_csv('train2.tsv', sep='\t', header=None)
df_test = pd.read_csv('test2.tsv', sep='\t', header=None)

Y_train = np.array(df_train[0].values.astype(int))
Y_test = np.array(df_test[0].values.astype(int))
X_train = np.array(df_train.iloc[:,2:].values.astype(float))
X_test = np.array(df_test.iloc[:,2:].values.astype(float))

Y_train = Y_train.astype(np.int)
Y_test = Y_test.astype(np.int)

Y_train = np_utils.to_categorical(Y_train, 2)
Y_test = np_utils.to_categorical(Y_test, 2)

print (X_train.shape)
print (Y_train.shape)
print (X_test.shape)
print (Y_test.shape)

/home/user/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


(9498, 5160)
(9498, 2)
(2574, 5160)
(2574, 2)


Let's consider how the simple Newral Network(NN), Multilayer Perceptron (MLP), with 3 hidden layers, constructed by Keras, can help us to solve this problem.

As we have hidden layers - this would be a Deep Neural Network. 
Also, we can see, that we need to have 5160 neurons in the input layer, as this is the size of our vector X and 2 neurons in the last layer - as this is the size of our target.
You can read, for example, [here](https://en.wikipedia.org/wiki/Multilayer_perceptron) some more information about MLP structure. 

<center>
<img src= "http://i66.tinypic.com/2d6tsm.jpg
" style="height:250px">  

The core data structure of Keras is a  **_model_** - a way to organize layers. The simplest type of model is the **_Sequential_** model, a linear stack of layers, which is appropriate for MLP construction (for more complex architectures, you should use the Keras functional API, which allows to build arbitrary graphs of layers).

In [2]:
import keras
from keras import Sequential
model1 = Sequential() 

After the type of model is defined - we need to consistently add layers -**_Dense_**. Stacking layers is as easy as **_.add()_**.

While adding the layer we need to define the ***nput_dim*** number of neurons and activation functionsa. For the fist layer we also need to add the dimention of X vectors. In our case this is 5,160. The last layer consists on 2 neurons exactly as our target vestors Y_train and Y_test do.

We can choose the **_Activation_** functions as well as the _number of neurons_ for each certain layer which look appropriate for the base line and later tune this if necessary.

In [3]:
from keras.layers import Activation, Dense

model1.add(Dense(64, input_dim=5160))
model1.add(Activation('relu'))

model1.add(Dense(64))
model1.add(Activation('sigmoid'))

model1.add(Dense(128))
model1.add(Activation('tanh'))

model1.add(Dense(2))
model1.add(Activation('softmax'))

Once our model looks good, we need configure its learning process with **_.compile()_**.

Here we should also describe the **_loss_** function and **_metrics_** we want to use as well as **_optimizer_** (the type of the Gradient descent to be used) which seem appropriate in each particular case.

In [4]:
model1.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Now we can iterate on our training data in batches with the ***batch_size*** we want, where X_train and y_train are Numpy arrays just like in the Scikit-Learn API.
Also we can define the number of ***epochs*** (i.e. the max number of the full cycles of model's training). 
***Verbose=1*** just let us see the summary of the current stage of calcualtions.

We can also printing our model parameters using ***model.summary()***. 
It is also can be useful to see the shapes of X_train, y_train,X_test,y_test

Also, if we would be satisfied by the results of the model we can save the best one via ***callback_save*** option.
 
And there is an ***callback_early stop*** option to stop the training process when we don't have significant improvement during the certain number of epochs (***patience***). 
 
Now our first model is ready:

In [5]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

model1.summary()

print (X_train.shape)
print (Y_train.shape)
print (X_test.shape)
print (Y_test.shape)

callback_save       = ModelCheckpoint('best_model1.model1', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
callback_earlystop  = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')


model1.fit(  
    X_train,   
    Y_train,   
    batch_size=20,   
    epochs=10000,  
    verbose=1,
    validation_data=(X_test, Y_test), 
    callbacks=[callback_save, callback_earlystop]
)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                330304    
_________________________________________________________________
activation_1 (Activation)    (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
activation_2 (Activation)    (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               8320      
_________________________________________________________________
activation_3 (Activation)    (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 258       
__________

**So, we got a Baseline with Accuracy= 0.81585**. This happened on the 2nd epoch already.It looks cool, as we even didn't tune anythong yet!

Let's try to improve this result. For example, we can introduce **Dropout** - this is a kind of regularization for the Neral Networks.
The level of drop out is a probability of the exclusion the random choice neuron from the calculations. So, drop outs help to prevent the NN overfitting.  
Let's create the new model:

In [6]:
from keras.layers import Dropout

model2 = Sequential() 

model2.add(Dense(64, input_dim=5160))
model2.add(Activation('relu'))
model2.add(Dropout(0.3, seed=123))

model2.add(Dense(64))
model2.add(Activation('sigmoid'))
model2.add(Dropout(0.4, seed=123))

model2.add(Dense(128))
model2.add(Activation('tanh'))
model2.add(Dropout(0.5, seed=123))

model2.add(Dense(2))
model2.add(Activation('softmax'))


model2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model2.summary()

print (X_train.shape)
print (Y_train.shape)
print (X_test.shape)
print (Y_test.shape)

callback_save       = ModelCheckpoint('best_model2.model2', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
callback_earlystop  = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')


model2.fit(  
    X_train,   
    Y_train,   
    batch_size=20,   
    epochs=10000,  
    verbose=1,
    validation_data=(X_test, Y_test), 
    callbacks=[callback_save, callback_earlystop]
)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 64)                330304    
_________________________________________________________________
activation_5 (Activation)    (None, 64)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)                4160      
_________________________________________________________________
activation_6 (Activation)    (None, 64)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 128)               8320      
__________

Thud, adding the drop-outs we've **increased Accuracy on the test up to 0.85082**

We can also tune the rest of gyper-parameters like the *number of layers*, the *levels of drop-outs*, *activation functions*, *optimizer* etc.
For this purposes we can use, for example, another very friendly and easy-to-apply  - Hyperas library. The description with examples you can find [here](https://github.com/maxpumperla/hyperas).
As a result of such tuning we've got the following model configuration:

In [51]:
model3 = Sequential()
model3.add(Dense(64, input_dim=5160))
model3.add(Activation('relu'))
model3.add(Dropout(0.11729755246044238, seed=123))
    
model3.add(Dense(256))
model3.add(Activation('relu'))
model3.add(Dropout(0.8444244099007299,seed=123))

model3.add(Dense(1024))
model3.add(Activation('linear'))
model3.add(Dropout(0.41266207281071243,seed=123))

model3.add(Dense(256))
model3.add(Activation('relu'))
model3.add(Dropout(0.4844455237320119,seed=123))

model3.add(Dense(2))
model3.add(Activation('softmax'))

model3.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model3.summary()

print (X_train.shape)
print (Y_train.shape)
print (X_test.shape)
print (Y_test.shape)

callback_save       = ModelCheckpoint('best_model3.model3', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
callback_earlystop  = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')

model3.fit(  
    X_train,   
    Y_train,   
    batch_size=60,   
    epochs=10000,  
    verbose=1,
    validation_data=(X_test, Y_test), 
    callbacks=[callback_save, callback_earlystop]
)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_57 (Dense)             (None, 64)                330304    
_________________________________________________________________
activation_55 (Activation)   (None, 64)                0         
_________________________________________________________________
dropout_51 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_58 (Dense)             (None, 256)               16640     
_________________________________________________________________
activation_56 (Activation)   (None, 256)               0         
_________________________________________________________________
dropout_52 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_59 (Dense)             (None, 1024)              263168    
__________

######  Now, with tunned parameters we've managed to imporove Accuracy up to 0.86674

With Keras it is also possible to use  **L1/L2 weight regularization** which allow to apply penalties on layer parameters or layer activity during optimization. These penalties are incorporated in the loss function that the network optimizes.

In [57]:
from keras import regularizers 

model4 = Sequential()
model4.add(Dense(64, input_dim=5160,kernel_regularizer=regularizers.l2(0.0015), 
                 activity_regularizer=regularizers.l1(0.0015)))
model3.add(Activation('relu'))
model4.add(Dropout(0.11729755246044238, seed=123))
    
model4.add(Dense(256))
model4.add(Activation('relu'))
model4.add(Dropout(0.8444244099007299,seed=123))

model4.add(Dense(1024))
model4.add(Activation('linear'))
model4.add(Dropout(0.41266207281071243,seed=123))

model4.add(Dense(256))
model4.add(Activation('relu'))
model4.add(Dropout(0.4844455237320119,seed=123))

model4.add(Dense(2))
model4.add(Activation('softmax'))

model4.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model4.summary()

print (X_train.shape)
print (Y_train.shape)
print (X_test.shape)
print (Y_test.shape)

callback_save       = ModelCheckpoint('best_model4.model4', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
callback_earlystop  = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')

model4.fit(  
    X_train,   
    Y_train,   
    batch_size=60,   
    epochs=10000,  
    verbose=1,
    validation_data=(X_test, Y_test), 
    callbacks=[callback_save, callback_earlystop]
)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_87 (Dense)             (None, 64)                330304    
_________________________________________________________________
dropout_76 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_88 (Dense)             (None, 256)               16640     
_________________________________________________________________
activation_86 (Activation)   (None, 256)               0         
_________________________________________________________________
dropout_77 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_89 (Dense)             (None, 1024)              263168    
_________________________________________________________________
activation_87 (Activation)   (None, 1024)              0         
__________

So, we can see, that adding regualrization with the current coeffitients to the firs layer we've got just **Accuracy of 0.8392** which didn't improve the situation which means, that, as usual, they should be carefully tuned :) 

When we **want to use the best trained model** we got, we can just download it and apply on the data needed.
Let's see what we'll get on the test set:

In [21]:
from keras.models import load_model

model=load_model( 'best_model3.model3')
result = model2.predict_on_batch(X_test)
result[:5]

array([[9.9921489e-01, 7.8508107e-04],
       [9.9908841e-01, 9.1164949e-04],
       [1.9480076e-01, 8.0519927e-01],
       [9.9921489e-01, 7.8508107e-04],
       [8.9435852e-01, 1.0564152e-01]], dtype=float32)

You may also be interested to get a **list of all weight tensors** of the model, as Numpy arrays:

In [34]:
weights =model.get_weights()
weights[:1]

[array([[-0.03492968, -0.0405129 , -0.00745696, ..., -0.03112122,
         -0.02895394, -0.03320112],
        [-0.00709045,  0.02336991, -0.00662623, ..., -0.02707123,
          0.01238516,  0.00902536],
        [ 0.00911781, -0.02620855, -0.01481253, ..., -0.00985456,
         -0.02947772, -0.02592223],
        ...,
        [ 0.022276  , -0.02593126, -0.02306574, ..., -0.03842297,
          0.00451999, -0.02098652],
        [-0.03659435, -0.02750918, -0.00098653, ..., -0.04156897,
          0.02848635, -0.00270199],
        [ 0.03085619,  0.00692714,  0.00164401, ...,  0.02794782,
         -0.02217145,  0.03590545]], dtype=float32)]

Besides, you would propbably like to get the **model config** to re-use it on the future. This can be done as:

In [36]:
config= model.get_config()
config

[{'class_name': 'Dense',
  'config': {'activation': 'linear',
   'activity_regularizer': None,
   'batch_input_shape': (None, 5160),
   'bias_constraint': None,
   'bias_initializer': {'class_name': 'Zeros', 'config': {}},
   'bias_regularizer': None,
   'dtype': 'float32',
   'kernel_constraint': None,
   'kernel_initializer': {'class_name': 'VarianceScaling',
    'config': {'distribution': 'uniform',
     'mode': 'fan_avg',
     'scale': 1.0,
     'seed': None}},
   'kernel_regularizer': None,
   'name': 'dense_9',
   'trainable': True,
   'units': 64,
   'use_bias': True}},
 {'class_name': 'Activation',
  'config': {'activation': 'relu', 'name': 'activation_9', 'trainable': True}},
 {'class_name': 'Dropout',
  'config': {'name': 'dropout_4',
   'noise_shape': None,
   'rate': 0.11729755246044238,
   'seed': 123,
   'trainable': True}},
 {'class_name': 'Dense',
  'config': {'activation': 'linear',
   'activity_regularizer': None,
   'bias_constraint': None,
   'bias_initializer': {'c

So, the model can be *reinstantiated* from its config via:

In [38]:
model3 = Sequential.from_config(config)

For more model tuning opportunities proposed by Keras pls see [here](https://keras.io/)

What about the other type of the Neural Networks?
Yes, you can use the similar approach types to the layers' construction principles for LSTM, CNN and some other types of the Deep Neural Networks. For more details pls see [here](https://keras.io/)

The sources used: https://keras.io/, https://towardsdatascience.com/meet-artificial-neural-networks-ae5939b1dd3a, https://www.quantinsti.com/blog/installing-keras-python-r, https://livebook.manning.com/#!/book/deep-learning-with-python/chapter-7, https://github.com/hyperopt/hyperopt